# 03-01 : Training on 10% sample

In [1]:
import os

# go to the directory of the run script
os.chdir('../../')

In [2]:
import pandas as pd

## File Paths

In [3]:
model_path = "./models"
source_path = "/home/johnny/swan/code/dsm500-2023-oct/data/07_trans4rec/07-02_bundle_10"
data_path = "./data/subs_bundl/10"


train_file = "bundle_only_view_train_tr"
test_file = "bundle_only_view_test"

train_path = f"{data_path}/{train_file}.tsv" 
test_path = f"{data_path}/{test_file}.tsv" 

## Preprocessing

In [4]:
# load the source data
df_train_source = pd.read_parquet(f"{source_path}/{train_file}.parquet.gz", engine='pyarrow')
df_test_source = pd.read_parquet(f"{source_path}/{test_file}.parquet.gz", engine='pyarrow')

# rename the UserId to SessionId
df_train_source.rename(columns={'UserId': 'SessionId'}, inplace=True)
df_test_source.rename(columns={'UserId': 'SessionId'}, inplace=True)

# sort the data by SessionId and Timestamp
df_train_source.sort_values(by=['SessionId', 'Time'], inplace=True)
df_test_source.sort_values(by=['SessionId', 'Time'], inplace=True)

# save to tsv
df_train_source.to_csv(train_path, sep='\t', index=False)
df_test_source.to_csv(test_path, sep='\t', index=False)

## Create Run Script

In [6]:
def create_gru4rec_script(train_path:str,
                          test_path:str,
                          model_path:str,
                          pf:str,
                          m:str='1 5 10 20',
                          d:str='GPU:0') -> str:
    """
    Creates a script to run the GRU4Rec model
    """
    return f"{train_path} -t {test_path} -m {m} -pf {pf} -d {d} -s {model_path}"

## Coveo Params 

In [7]:
# best hyperparameters
paramfile = "./paramfiles/coveo_bprmax_shared_best.py"

# create the script
run_script = create_gru4rec_script(
    train_path=train_path,
    test_path=test_path,
    model_path=f"{model_path}/sub_bundl_10_coveo.pickle",

    pf=paramfile
)

print(run_script)

./data/subs_bundl/10/bundle_only_view_train_tr.tsv -t ./data/subs_bundl/10/bundle_only_view_test.tsv -m 1 5 10 20 -pf ./paramfiles/coveo_bprmax_shared_best.py -d GPU:0 -s ./models/sub_bundl_10_coveo.pickle


In [8]:
%run run.py $run_script

2024-02-07 03:47:43.993828: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 03:47:43.993857: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 03:47:43.994731: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-07 03:47:43.998093: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-07 03:47:44.490693: W tensorflow/compiler/tf2

Loaded parameters from file: /media/johnny/swan/code/GRU4Rec/paramfiles/coveo_bprmax_shared_best.py
Creating GRU4Rec model
SET   loss                    TO   bpr-max   (type: <class 'str'>)
SET   constrained_embedding   TO   True      (type: <class 'bool'>)
SET   embedding               TO   0         (type: <class 'int'>)
SET   elu_param               TO   1.0       (type: <class 'float'>)
SET   layers                  TO   [512]     (type: <class 'list'>)
SET   n_epochs                TO   10        (type: <class 'int'>)
SET   batch_size              TO   144       (type: <class 'int'>)
SET   dropout_p_embed         TO   0.35      (type: <class 'float'>)
SET   dropout_p_hidden        TO   0.0       (type: <class 'float'>)
SET   learning_rate           TO   0.05      (type: <class 'float'>)
SET   momentum                TO   0.4       (type: <class 'float'>)
SET   n_sample                TO   2048      (type: <class 'int'>)
SET   sample_alpha            TO   0.2       (type: <class 'f

2024-02-07 03:47:56.878320: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch1 --> loss: 0.382291 	(521.11s) 	[417.01 mb/s | 60048 e/s]
Epoch2 --> loss: 0.378574 	(517.93s) 	[419.57 mb/s | 60416 e/s]
Epoch3 --> loss: 0.377484 	(512.51s) 	[424.00 mb/s | 61055 e/s]
Epoch4 --> loss: 0.376963 	(513.44s) 	[423.24 mb/s | 60945 e/s]
Epoch5 --> loss: 0.376636 	(518.88s) 	[418.80 mb/s | 60306 e/s]
Epoch6 --> loss: 0.376400 	(514.63s) 	[422.26 mb/s | 60804 e/s]
Epoch7 --> loss: 0.376197 	(507.93s) 	[427.83 mb/s | 61606 e/s]
Epoch8 --> loss: 0.376053 	(518.04s) 	[419.48 mb/s | 60404 e/s]
Epoch9 --> loss: 0.375946 	(509.08s) 	[426.86 mb/s | 61467 e/s]
Epoch10 --> loss: 0.375844 	(513.73s) 	[423.00 mb/s | 60910 e/s]
Total training time: 5153.05s
Saving trained model to: ./models/sub_bundl_10_coveo.pickle
Loading test data...
Loading data from TAB separated file: ./data/subs_bundl/10/bundle_only_view_test.tsv
Starting evaluation (cut-off=[1, 5, 10, 20], using standard mode for tiebreaking)
Using existing item ID map
The dataframe is already sorted by SessionId, Time
Eva

## Diginetica Params 

In [10]:
# create the script
run_script = create_gru4rec_script(
    train_path=train_path,
    test_path=test_path,
    model_path=f"{model_path}/sub_bundl_10_diginetica.pickle",
    pf="./paramfiles/diginetica_bprmax_shared_best.py"
)

print(run_script)

./data/subs_bundl/10/bundle_only_view_train_tr.tsv -t ./data/subs_bundl/10/bundle_only_view_test.tsv -m 1 5 10 20 -pf ./paramfiles/diginetica_bprmax_shared_best.py -d GPU:0 -s ./models/sub_bundl_10_diginetica.pickle


In [11]:
%run run.py $run_script

Loaded parameters from file: /media/johnny/swan/code/GRU4Rec/paramfiles/diginetica_bprmax_shared_best.py
Creating GRU4Rec model
SET   loss                    TO   bpr-max   (type: <class 'str'>)
SET   constrained_embedding   TO   True      (type: <class 'bool'>)
SET   embedding               TO   0         (type: <class 'int'>)
SET   elu_param               TO   1.0       (type: <class 'float'>)
SET   layers                  TO   [512]     (type: <class 'list'>)
SET   n_epochs                TO   10        (type: <class 'int'>)
SET   batch_size              TO   128       (type: <class 'int'>)
SET   dropout_p_embed         TO   0.5       (type: <class 'float'>)
SET   dropout_p_hidden        TO   0.3       (type: <class 'float'>)
SET   learning_rate           TO   0.05      (type: <class 'float'>)
SET   momentum                TO   0.15      (type: <class 'float'>)
SET   n_sample                TO   2048      (type: <class 'int'>)
SET   sample_alpha            TO   0.3       (type: <cla